In [1]:
# Manas Pratim Biswas
# 002011001025

In [2]:
import collections
import copy
import numpy as np
import sys

In [3]:
with open('initialState.txt') as f:
    lines = f.readlines()
    
initialState = []
finalState = []

for line in lines:
    row = [int(i) for i in line.split()]
    initialState.append(row)
    
with open('finalState.txt') as g:
    lines = g.readlines()
    
for line in lines:
    row = [int(i) for i in line.split()]
    finalState.append(row)

In [4]:
def printLine():
    print("----------------------------------\n")

In [5]:
class Node:
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action


class StackFrontier:
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any((node.state[0] == state[0]).all() for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("Empty Frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node


class QueueFrontier(StackFrontier):
    def remove(self):
        if self.empty():
            raise Exception("Empty Frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node


class Puzzle:
    def __init__(self, start, startIndex, goal, goalIndex):
        self.start = [start, startIndex]
        self.goal = [goal, goalIndex] 
        self.solution = None

    def neighbors(self, state):
        mat, (row, col) = state
        results = []

        if row > 0:
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row - 1][col]
            mat1[row - 1][col] = 0
            results.append(('UP', [mat1, (row - 1, col)]))
        if col > 0:
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row][col - 1]
            mat1[row][col - 1] = 0
            results.append(('LEFT', [mat1, (row, col - 1)]))
        if row < 2:
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row + 1][col]
            mat1[row + 1][col] = 0
            results.append(('DOWN', [mat1, (row + 1, col)]))
        if col < 2:
            mat1 = np.copy(mat)
            mat1[row][col] = mat1[row][col + 1]
            mat1[row][col + 1] = 0
            results.append(('RIGHT', [mat1, (row, col + 1)]))

        return results

    def print(self):
        solution = self.solution if self.solution is not None else None
        print("Initial State:-\n", self.start[0], "\n")
        print("Final State:-\n",  self.goal[0], "\n")
        print("\nStates Explored: ", self.num_explored, "\n")
        print("Solution:-\n ")
        for action, cell in zip(solution[0], solution[1]):
            print("Move:- ", action, "\n", cell[0], "\n")
        print("The Final State is Reached!")

    def does_not_contain_state(self, state):
        for st in self.explored:
            if (st[0] == state[0]).all():
                return False
        return True

    def solve(self):
        self.num_explored = 0

        start = Node(state=self.start, parent=None, action=None)
        frontier = QueueFrontier()
        frontier.add(start)

        self.explored = [] 

        while True:
            if frontier.empty():
                raise Exception("No Solution!")

            node = frontier.remove()
            self.num_explored += 1

            if (node.state[0] == self.goal[0]).all():
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                    actions.reverse()
                cells.reverse()
                self.solution = (actions,  cells)
                return

            self.explored.append(node.state)

            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and self.does_not_contain_state(state):
                    child = Node(state=state, parent=node, action=action)
                    frontier.add(child)


In [6]:
def generateAllStates(board) -> int:
        m = len(board)
        n = len(board[0])        
        moves = 0
        q = collections.deque([]) 
        visited = set()
       
        
        def getNeighbours(i,j,board):
            ans = []
            for x,y in [(1,0),(0,1),(-1,0),(0,-1)]:
                nx = i+x
                ny = j+y        
                
                if nx < 0 or nx >= m or ny < 0 or ny >= n:
                    continue
                board[i][j],board[nx][ny] = board[nx][ny], board[i][j]
                ans.append((nx,ny,copy.deepcopy(board)))
                board[nx][ny],board[i][j] = board[i][j],board[nx][ny]  
            return ans          
        
        for i in range(m):
            for j in range(n):
                if board[i][j]==0:
                    q.append((i,j,board.copy()))
        ans = -1
        temporaryBoard=[]
        while q:
            print("Current Level :- ",moves+1)
            print("All Possible States :- ")
            printLine()
            for _ in range(len(q)):
                curri,currj,currBoard=q.popleft()
                visited.add(str(currBoard))
                
                if currBoard==finalState: 
                    return moves
                
                neighbours = getNeighbours(curri,currj,currBoard)
               
                
                for x,y,board in neighbours:
                    if str(board) not in visited:
                        npBoard = np.array(board)
                        print(npBoard)
                        print()
                        temporaryBoard.append(board)
                        q.append((x,y,board))
                        
            moves += 1
        
        return -1;

In [7]:
initialGrid = np.array(initialState)
finalGrid = np.array(finalState)
initialX = -1
initialY = -1
finalX = -1
finalY = -1

In [8]:
for i in range(3):
    for j in range(3):
        if(initialState[i][j]==0):
            initialX = i
            initialY = j
            
for i in range(3):
    for j in range(3):
        if(finalState[i][j]==0):
            finalX = i
            finalY = j

In [9]:
initialIndex = (initialX,initialY)
finalIndex = (finalX,finalY)

In [10]:
minimumSteps = generateAllStates(initialState)

Current Level :-  1
All Possible States :- 
----------------------------------

[[1 2 3]
 [8 6 4]
 [7 0 5]]

[[1 2 3]
 [8 4 0]
 [7 6 5]]

[[1 0 3]
 [8 2 4]
 [7 6 5]]

[[1 2 3]
 [0 8 4]
 [7 6 5]]

Current Level :-  2
All Possible States :- 
----------------------------------

[[1 2 3]
 [8 6 4]
 [7 5 0]]

[[1 2 3]
 [8 6 4]
 [0 7 5]]

[[1 2 3]
 [8 4 5]
 [7 6 0]]

[[1 2 0]
 [8 4 3]
 [7 6 5]]

[[1 3 0]
 [8 2 4]
 [7 6 5]]

[[0 1 3]
 [8 2 4]
 [7 6 5]]

[[1 2 3]
 [7 8 4]
 [0 6 5]]

[[0 2 3]
 [1 8 4]
 [7 6 5]]

Current Level :-  3
All Possible States :- 
----------------------------------

[[1 2 3]
 [8 6 0]
 [7 5 4]]

[[1 2 3]
 [0 6 4]
 [8 7 5]]

[[1 2 3]
 [8 4 5]
 [7 0 6]]

[[1 0 2]
 [8 4 3]
 [7 6 5]]

[[1 3 4]
 [8 2 0]
 [7 6 5]]

[[8 1 3]
 [0 2 4]
 [7 6 5]]

[[1 2 3]
 [7 8 4]
 [6 0 5]]

[[2 0 3]
 [1 8 4]
 [7 6 5]]

Current Level :-  4
All Possible States :- 
----------------------------------

[[1 2 0]
 [8 6 3]
 [7 5 4]]

[[1 2 3]
 [8 0 6]
 [7 5 4]]

[[1 2 3]
 [6 0 4]
 [8 7 5]]

[[0 2 3]
 [1 

In [11]:
puzzle = Puzzle(initialGrid,initialIndex,finalGrid,finalIndex)
puzzle.solve()
puzzle.print()
print("The MINIMUM number of steps to reach the final state is :- ",minimumSteps)

Initial State:-
 [[1 2 3]
 [8 0 4]
 [7 6 5]] 

Final State:-
 [[2 8 1]
 [0 4 3]
 [7 6 5]] 


States Explored:  358 

Solution:-
 
Move:-  UP 
 [[1 0 3]
 [8 2 4]
 [7 6 5]] 

Move:-  DOWN 
 [[0 1 3]
 [8 2 4]
 [7 6 5]] 

Move:-  RIGHT 
 [[8 1 3]
 [0 2 4]
 [7 6 5]] 

Move:-  LEFT 
 [[8 1 3]
 [2 0 4]
 [7 6 5]] 

Move:-  DOWN 
 [[8 1 3]
 [2 4 0]
 [7 6 5]] 

Move:-  LEFT 
 [[8 1 0]
 [2 4 3]
 [7 6 5]] 

Move:-  UP 
 [[8 0 1]
 [2 4 3]
 [7 6 5]] 

Move:-  RIGHT 
 [[0 8 1]
 [2 4 3]
 [7 6 5]] 

Move:-  LEFT 
 [[2 8 1]
 [0 4 3]
 [7 6 5]] 

The Final State is Reached!
The MINIMUM number of steps to reach the final state is :-  9
